# Data Preprocessing

## Import Library

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## 데이터 로드

In [2]:
path = '../data/merged-traffic.csv'
filename = os.path.basename(path)
df = pd.read_csv(path)

## 극소수 클래스 제거 (전체 비중 0.1% 미만)

In [3]:
# 컬럼명 공백 제거
df.columns = df.columns.str.strip()
df['Label'].value_counts()

Label
BENIGN                        1454613
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [4]:
total_rows = len(df)
label_counts = df['Label'].value_counts()

threshold = total_rows * 0.001  # 기준: 전체의 0.1% 미만인 클래스
rare_labels = label_counts[label_counts < threshold].index.tolist()

print(f"[INFO] Rare classes to be removed (count < {threshold:.0f}):")
for label in rare_labels:
    print(f"- {label}: {label_counts[label]} rows")

df = df[~df['Label'].isin(rare_labels)].reset_index(drop=True)

[INFO] Rare classes to be removed (count < 2012):
- Bot: 1966 rows
- Web Attack � Brute Force: 1507 rows
- Web Attack � XSS: 652 rows
- Web Attack � Sql Injection: 21 rows
- Heartbleed: 11 rows


In [5]:
df['Label'].value_counts()

Label
BENIGN              1454613
DoS Hulk             231073
PortScan             158930
DDoS                 128027
DoS GoldenEye         10293
FTP-Patator            7938
SSH-Patator            5897
DoS slowloris          5796
DoS Slowhttptest       5499
Name: count, dtype: int64

## Binary 라벨링 (악성 트래픽: 1, 정상: 0)

In [6]:
df['Label_binary'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
df['Label_binary'].value_counts()

Label_binary
0    1454613
1     553453
Name: count, dtype: int64

## 불필요한 열 제거
- 식별자(Identifiers) 또는 고유값
- 목표값과 직접적인 관련이 있는 열 (Leakage Feature)
- 상수 값 (Constant Features)
- 결측치가 너무 많은 열
- 중복 정보 or 높은 상관관계를 가지는 열
- 텍스트, 로그, 비정형 필드

### 상수값 데이터 제거

In [7]:
low_variance_cols = [col for col in df.columns if df[col].nunique() <= 1]
df = df.drop(columns=low_variance_cols)
print(f'removed_low_variance_cols: {low_variance_cols}')

removed_low_variance_cols: ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']


### 결측 비율이 90% 이상인 데이터 제거

In [8]:
missing_cols = df.columns[df.isnull().mean() > 0.9]
df = df.drop(columns=missing_cols)
print(f'removed_missing_cols: {missing_cols}')

removed_missing_cols: Index([], dtype='object')


### 목표값과 직접적인 관련이 있는 데이터 제거

In [9]:
leakage_cols = ['Label']
df = df.drop(columns=leakage_cols)
print(f'removed_leakage_cols: {leakage_cols}')

removed_leakage_cols: ['Label']


### 높은 상관관계 데이터 제거 (상관계수 0.95 이상)

**상관계수 0.95 이상 쌍 확인**

In [10]:
# 수치형 데이터만 추출
df_numeric = df.select_dtypes(include=[np.number])

# 상관계수 행렬 계산 (절댓값 기준)
corr_matrix = df_numeric.corr().abs()

# 상삼각 행렬 추출 (중복 제거용)
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 0.95 이상인 쌍 추출
high_corr_pairs = []

for col in upper_tri.columns:
    for row in upper_tri.index:
        corr_value = upper_tri.loc[row, col]
        if pd.notnull(corr_value) and corr_value >= 0.95:
            high_corr_pairs.append((row, col, round(corr_value, 4)))

# 내림차순 정렬
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 결과 출력
for a, b, corr in high_corr_pairs:
    print(f"{a:30} <--> {b:30} : {corr}")

Fwd PSH Flags                  <--> SYN Flag Count                 : 1.0
Fwd Packet Length Mean         <--> Avg Fwd Segment Size           : 1.0
Bwd Packet Length Mean         <--> Avg Bwd Segment Size           : 1.0
Fwd Header Length              <--> Fwd Header Length.1            : 1.0
Total Fwd Packets              <--> Subflow Fwd Packets            : 1.0
Total Length of Fwd Packets    <--> Subflow Fwd Bytes              : 1.0
Total Backward Packets         <--> Subflow Bwd Packets            : 1.0
Total Length of Bwd Packets    <--> Subflow Bwd Bytes              : 1.0
Total Fwd Packets              <--> Total Backward Packets         : 0.9991
Total Backward Packets         <--> Subflow Fwd Packets            : 0.9991
Total Fwd Packets              <--> Subflow Bwd Packets            : 0.9991
Subflow Fwd Packets            <--> Subflow Bwd Packets            : 0.9991
Flow Duration                  <--> Fwd IAT Total                  : 0.9988
Flow IAT Max                   <--> 

**제거할 feature 추출**

In [11]:
features_to_drop = set()
features_to_keep = set()
keywords = ['.1', 'Subflow', 'Avg', 'Segment', 'Bytes', 'Bulk']
already_dropped = set()

for col in upper_tri.columns:
    for row in upper_tri.index:
        corr_value = upper_tri.loc[row, col]
        if pd.notnull(corr_value) and corr_value >= 0.95:
            # 이미 제거된 컬럼은 무시
            if row in features_to_drop or col in features_to_drop:
                continue
            
            # 1. 파생/복제 이름 기준
            if any(kw in row for kw in keywords) and not any(kw in col for kw in keywords):
                drop_feature = row
                keep_feature = col
            elif any(kw in col for kw in keywords) and not any(kw in row for kw in keywords):
                drop_feature = col
                keep_feature = row
            
            # 2. 결측치가 많은 쪽 제거
            elif df[row].isnull().sum() > df[col].isnull().sum():
                drop_feature = row
                keep_feature = col
            elif df[col].isnull().sum() > df[row].isnull().sum():
                drop_feature = col
                keep_feature = row
            
            # 3. 고유값이 적은 쪽 제거
            elif df[row].nunique() < df[col].nunique():
                drop_feature = row
                keep_feature = col
            elif df[col].nunique() < df[row].nunique():
                drop_feature = col
                keep_feature = row
            
            # 4. 나머지는 col 제거(순서상 일관성 위해)
            else:
                drop_feature = col
                keep_feature = row
            features_to_drop.add(drop_feature)
            features_to_keep.add(keep_feature)

df = df.drop(columns=list(features_to_drop))
print(f'removed_high_correlated_cols: {features_to_drop}')

removed_high_correlated_cols: {'Total Fwd Packets', 'Idle Min', 'Total Backward Packets', 'SYN Flag Count', 'Max Packet Length', 'Fwd Packet Length Max', 'Subflow Bwd Bytes', 'Bwd Packet Length Max', 'Idle Max', 'Fwd Packets/s', 'ECE Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Idle Mean', 'Subflow Bwd Packets', 'Fwd IAT Total', 'Subflow Fwd Bytes', 'Fwd Header Length.1', 'Fwd IAT Max'}


### 남아있는 필드명 확인

In [12]:
columns_list = df.columns.tolist()
columns_list

['Destination Port',
 'Flow Duration',
 'Total Length of Fwd Packets',
 'Total Length of Bwd Packets',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Flow Bytes/s',
 'Flow Packets/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Min',
 'Bwd IAT Total',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Fwd Header Length',
 'Bwd Header Length',
 'Bwd Packets/s',
 'Min Packet Length',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'FIN Flag Count',
 'RST Flag Count',
 'PSH Flag Count',
 'ACK Flag Count',
 'URG Flag Count',
 'Down/Up Ratio',
 'Init_Win_bytes_forward',
 'Init_Win_bytes_backward',
 'act_data_pkt_fwd',
 'min_seg_size_forward',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Std',
 'Label_binary']

## 결측치 처리
- 수치형: 평균
- 범주형: 최빈값

In [13]:
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 변환

for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].mean(), inplace=True)

print(df.isnull().sum().sum())  # 전체 NaN 개수
print(np.isinf(df.select_dtypes(include=[np.number])).sum().sum())  # 전체 inf 개수

/var/folders/gt/m7w4jt2s7774gl7rdr6rwclm0000gn/T/ipykernel_3596/3983051279.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


0
0


## 이상치 처리

In [14]:
# 수치형 컬럼만 선택 (정규화/라벨 컬럼 등은 제외)
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Label', 'Label_binary']]

# IQR 계산 및 클리핑
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# 각 feature별로 lower, upper 구해서 클리핑
for col in numerical_cols:
    lower = Q1[col] - 1.5 * IQR[col]
    upper = Q3[col] + 1.5 * IQR[col]
    df[col] = df[col].clip(lower=lower, upper=upper)

# 이상치 처리 결과 확인
df[numerical_cols].describe()

,Destination Port,Flow Duration,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,...,Down/Up Ratio,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std
count,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,...,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2.008066e+06,2008066.0,2008066.0,2008066.0,2008066.0,2008066.0
mean,3.331657e+02,3.134571e+06,1.734911e+02,6.031749e+02,1.546568e+01,3.512839e+01,2.560706e+01,3.632248e+01,1.575549e+02,1.344154e+02,...,6.732279e-01,4.435611e+03,1.200213e+02,1.470900e+00,2.612816e+01,0.0,0.0,0.0,0.0,0.0
std,3.865595e+02,5.187479e+06,2.424486e+02,8.769449e+02,2.116086e+01,3.325743e+01,4.163901e+01,5.470584e+01,1.977911e+02,2.276242e+02,...,5.538132e-01,7.438205e+03,1.839667e+02,1.676465e+00,6.697132e+00,0.0,0.0,0.0,0.0,0.0
min,0.000000e+00,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,2.000000e+00,0.0,0.0,0.0,0.0,0.0
25%,5.300000e+01,1.640000e+02,6.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,5.500000e+00,0.000000e+00,...,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,2.000000e+01,0.0,0.0,0.0,0.0,0.0
50%,8.000000e+01,3.768850e+04,6.200000e+01,1.260000e+02,0.000000e+00,3.400000e+01,0.000000e+00,0.000000e+00,7.400000e+01,0.000000e+00,...,1.000000e+00,2.510000e+02,0.000000e+00,1.000000e+00,2.000000e+01,0.0,0.0,0.0,0.0,0.0
75%,4.430000e+02,5.245993e+06,2.930000e+02,8.610000e+02,3.500000e+01,5.000000e+01,4.036998e+01,7.000000e+01,2.147500e+02,2.173724e+02,...,1.000000e+00,8.192000e+03,2.350000e+02,2.000000e+00,3.200000e+01,0.0,0.0,0.0,0.0,0.0
max,1.028000e+03,1.311474e+07,7.235000e+02,2.143500e+03,8.750000e+01,1.160000e+02,1.009250e+02,1.750000e+02,5.286250e+02,5.434309e+02,...,2.500000e+00,2.048150e+04,5.890000e+02,5.000000e+00,5.000000e+01,0.0,0.0,0.0,0.0,0.0


## 수치형 정규화
- 수치형 데이터를 0~1 범위로 스케일링

In [15]:
# 수치형 컬럼 추출 (라벨/정답 컬럼은 제외)
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Label', 'Label_binary']]

# 정규화 객체 생성 및 fit_transform
scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# 정규화 결과 확인
df[numerical_cols].head()

,Destination Port,Flow Duration,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,...,Down/Up Ratio,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std
0,1.000000,1.000000,1.000000,0.537439,0.00,1.000000,1.0,0.411429,0.136202,0.0,...,0.0,0.018455,1.0,1.0,0.625,0.0,0.0,0.0,0.0,0.0
1,0.378405,1.000000,1.000000,1.000000,0.00,1.000000,1.0,1.000000,0.597777,0.0,...,0.0,0.046674,1.0,1.0,0.625,0.0,0.0,0.0,0.0,0.0
2,0.000000,1.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
3,1.000000,0.007636,0.851417,0.000000,0.32,0.241379,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,1.0,0.625,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.004176,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0


## 결과 저장

In [16]:
save_path = f'../data/preprocessed-{filename}'
df.to_csv(save_path, index=False)